In [13]:
import csv
questionsfile = '/files/faust/COMPETITION/ppdai/question.csv'
questions  = {}
with open(questionsfile) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        qtext  = {}
        qtext['cwords'] = row['words'].strip().split(' ')
        qtext['cchars'] = row['chars'].strip().split(' ')
        questions[row['qid']] = qtext

In [14]:
print(len(questions))
iwords = questions['Q000012']['cwords']
# iwords = questions['Q000012']['chars']
print(iwords)
print(type(iwords))

728428
['W10139', 'W15218', 'W12566', 'W01490']
<class 'list'>


In [15]:
import pickle
with open('/files/faust/COMPETITION/ppdai/questions.pkl', 'wb') as f:
    pickle.dump(questions, f)

In [ ]:
def ngramextend(wlist, n):
    result = []
    for i in range(len(wlist)):
        for j in range(1, n+1):
            if (i+j) <= len(wlist):
                merged = ''.join(wlist[i:i+j])
                result.append(merged)
    return result
    
ws = ['W10139', 'W15218', 'W12566', 'W01490']
result = ngramextend(ws, 3)
print(result)

In [ ]:
questions_ngram = {}
for q_id, wc in questions.items():
    ori_w = wc['words']
    ori_c = wc['chars']
    qtext = {}
    qtext['cwords'] = ngramextend(ori_w, 3)
    qtext['cchars'] = ngramextend(ori_c, 5)
    questions_ngram[q_id] = qtext

In [ ]:
import pickle
with open('/files/faust/COMPETITION/ppdai/questions_ngram_extend.pkl', 'wb') as f:
    pickle.dump(questions_ngram, f)

In [16]:
import pickle
# questions = pickle.load(open('/files/faust/COMPETITION/ppdai/questions_ngram_extend.pkl', 'rb'))
questions = pickle.load(open('/files/faust/COMPETITION/ppdai/questions.pkl', 'rb'))

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus_words = [' '.join(wc['cwords']) for _, wc in questions.items()]
vec_cw = TfidfVectorizer()
vec_cw.fit(corpus_words)

corpus_chars = [' '.join(wc['cchars']) for _, wc in questions.items()]
vec_cc = TfidfVectorizer()
vec_cc.fit(corpus_chars)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [17]:
word_count = {}

for _, wc in questions.items():
    for word in wc['cwords']:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1

In [19]:
print(len(word_count))
word_count_filt = {}
for w,f in word_count.items():
    if f > 1:
        word_count_filt[w] = f
print(len(word_count_filt))
wdict = [w for w,_ in word_count_filt.items()]
wdict = dict((k,v) for v,k in enumerate(wdict))
print(wdict)

20890
11202


In [46]:
import numpy as np
def bag_of_words(sentence):
    vector = np.zeros((len(wdict) + 1,), dtype=np.int32)
    for w in sentence:
        if w in wdict:
            vector[wdict[w]] = 1
        else:
            vector[-1] = 1
    return vector

# MODEL

In [51]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

indim = len(wdict) + 1
model = Sequential()
model.add(Dense(units=1024, input_dim=indim))
model.add(Activation("tanh"))
model.add(Dense(units=256))
model.add(Activation("tanh"))
model.add(Dense(units=1024))
model.add(Activation("tanh"))
model.add(Dense(units=indim))
model.add(Activation("sigmoid"))
model.compile(loss='mse', optimizer='adam', metrics=['mse'])


## read data

In [43]:
def getvector_with_id(qid):
    words = questions[qid]['cwords']
    vector =bag_of_words(words)
    return vector

# read train pair
data = []
import csv
trainfile = '/files/faust/COMPETITION/ppdai/train.csv'
with open(trainfile) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        data.append((row['q1'], row['q2'], row['label']))
print(len(data))

254386


## dataset split

In [36]:
import random
random.shuffle(data)
traindata = data[:int(len(data)*0.9)]
testdata  = data[int(len(data)*0.9):]
print(len(traindata))
print(len(testdata))

228947
25439


In [ ]:
## for translation-based/alignment
traindata_pos = [q for q in traindata if q[2] == '1']
testdata_pos = [q for q in testdata if q[2] == '1']
print(len(traindata_pos))

qidlist = list(questions.keys())
qid_sample = random.sample(qidlist, len(traindata_pos))
traindata_copy = [(qid,qid,1) for qid in qid_sample]

traindata_pos.extend(traindata_copy)
random.shuffle(traindata_pos)
print(len(traindata_pos))

In [47]:
train_x = np.array([getvector_id(q[0]) for q in traindata_copy])
train_y = np.array([getvector_id(q[1]) for q in traindata_copy])
test_x = np.array([getvector_id(q[0]) for q in testdata_pos])
test_y = np.array([getvector_id(q[1]) for q in testdata_pos])

118835
237670


In [52]:
EPOCH = 20
BATCH_SIZE = 64
LEARNING_RATE = 0.025
model.fit(train_x, train_y, epochs=10, batch_size=64)

Epoch 1/20
  9600/118835 [=>............................] - ETA: 49s - loss: 4.9849e-04 - mean_squared_error: 4.9923e-0